# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

Выберете источник данных, который вам симпатичен. По умолчанию, предлагаем использовать данные о погоде в Москве: 
https://www.gismeteo.ru/weather-moscow-4368/. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

В рамках домашнего задания буду брать курсы доллара и евро с сайта ЦБ РФ

In [1]:
import pandas as pd
import re
import urllib.request
import numpy as np

from scrapy.selector import Selector
from scrapy.utils.markup import remove_tags

In [2]:
def get_currency(date):
    url = 'http://www.cbr.ru/currency_base/daily.aspx?date_req=' + str(date)
    req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
    html = urllib.request.urlopen(url)
    html = html.read()
    usd = remove_tags(str(Selector(text=html).xpath('//*[@id="content"]/div/table/tbody/tr[12]/td[5]').extract()))
    eur = remove_tags(str(Selector(text=html).xpath('//*[@id="content"]/div/table/tbody/tr[13]/td[5]').extract()))
    return round(float(usd[2:-2].replace(',', '.')), 2), round(float(eur[2:-2].replace(',', '.')), 2)

In [3]:
date = '23.12.2017'
usd, eur = get_currency(date)
print('Currencies on {}:\nusd: {}\neur: {}'.format(date, usd, eur))

Currencies on 23.12.2017:
usd: 58.32
eur: 69.13


## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [4]:
import pymorphy2
def soglasovanie_slov(number, word):
    morph = pymorphy2.MorphAnalyzer()
    word_0form = morph.parse(word)[0] 
    return word_0form.make_agree_with_number(number).word

In [5]:
def show_currencies(date):
    usd, eur = get_currency(date)
    print('Курс USD (ЦБ РФ) на {}: {} {} {:.0f} {}'.format(date, int(usd), soglasovanie_slov(int(usd), 'рубль'), \
                                                               int(round(usd-int(usd), 2)*100), soglasovanie_slov(int(round(usd-int(usd), 2)*100), 'копейка')))
    print('Курс EUR (ЦБ РФ) на {}: {} {} {:.0f} {}'.format(date, int(eur), soglasovanie_slov(int(eur), 'рубль'), \
                                                               int(round(eur-int(eur), 2)*100), soglasovanie_slov(int(round(eur-int(eur), 2)*100), 'копейка')))
show_currencies('20.12.2013')

Курс USD (ЦБ РФ) на 20.12.2013: 45 рублей 6 копеек
Курс EUR (ЦБ РФ) на 20.12.2013: 52 рубля 93 копейки


##### Часть 3 (необязательная)
Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели (например, "сегодня похолодало на Х градусов по сравнению со вчера").

In [6]:
from datetime import datetime, timedelta
ts = datetime.today()
today = ts.strftime('%d.%m.%Y')
yesterday = (ts - timedelta(1)).strftime('%d.%m.%Y')
tomorrow = (ts + timedelta(1)).strftime('%d.%m.%Y')

In [7]:
def show_currencies_chng(date1, date2):
    if date1 > date2:
        print('Вторая дата должна быть позже первой!')
        return 0
    usd1, eur1 =  get_currency(date1)
    usd2, eur2 = get_currency(date2)
    if int(usd2*100) == int(usd1*100):
        print('С {} по {} курс доллара (USD) не изменился'.format(date1, date2))
    elif usd2 > usd1:    
        print('С {} по {} курс доллара (USD) вырос на {} {} {} {}'\
              .format(date1, date2, int(usd2-usd1), soglasovanie_slov(int(usd2-usd1), 'рубль'),\
                      (round(usd2-usd1, 2)-int(usd2-usd1))*100, soglasovanie_slov(int((round(usd2-usd1, 2)-int(usd2-usd1))*100), 'копейка')))
    elif usd2 < usd1:
        print('С {} по {} курс доллара (USD) упал на {} {} {} {}'\
              .format(date1, date2, int(usd1-usd2), soglasovanie_slov(int(usd1-usd2), 'рубль'),\
                      int((round(usd1-usd2, 2)-int(usd1-usd2))*100), soglasovanie_slov(int((round(usd1-usd2, 2)-int(usd1-usd2))*100), 'копейка')))
    
    if int(eur2*100) == int(eur1*100):
        print('С {} по {} курс евро (EUR) не изменился'.format(date1, date2))
    elif eur2 > eur1:    
        print('С {} по {} курс евро (EUR) вырос на {} {} {} {}'\
              .format(date1, date2, int(eur2-eur1), soglasovanie_slov(int(eur2-eur1), 'рубль'),\
                      (round(eur2-eur1, 2)-int(eur2-eur1))*100, soglasovanie_slov(int(((eur2-eur1)-int(eur2-eur1))*100), 'копейка')))
    elif eur2 < eur1:
        print('С {} по {} курс евро (EUR) упал на {} {} {} {}'\
              .format(date1, date2, int(eur1-eur2), soglasovanie_slov(int(eur1-eur2), 'рубль'),\
                      int((round(eur1-eur2, 2)-int(eur1-eur2))*100), soglasovanie_slov(int(((eur1-eur2)-int(eur1-eur2))*100), 'копейка')))
    
show_currencies_chng(today, tomorrow)

show_currencies(today)
show_currencies(tomorrow)

С 25.12.2017 по 26.12.2017 курс доллара (USD) упал на 0 рублей 12 копеек
С 25.12.2017 по 26.12.2017 курс евро (EUR) упал на 0 рублей 4 копейки
Курс USD (ЦБ РФ) на 25.12.2017: 58 рублей 32 копейки
Курс EUR (ЦБ РФ) на 25.12.2017: 69 рублей 13 копеек
Курс USD (ЦБ РФ) на 26.12.2017: 58 рублей 20 копеек
Курс EUR (ЦБ РФ) на 26.12.2017: 69 рублей 9 копеек
